<a href="https://colab.research.google.com/github/CharlesCarpes/EnvioApostaAtumatizadaBetFair/blob/main/EnviaApostaBetFair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Código para gerar lista de jogos e enviar aposta

## Realiza conexão com a BetFair

In [ ]:
# Importanto pacotes
import datetime
from datetime import datetime, timedelta
import pandas as pd
import betfairlightweight
import os
import time
import dotenv

In [ ]:
#Aqui você vai colocar seu dados pessoas!
#cuidado para não compartilhar seu código com esses dados visíveis

email = "seu_email_betfair@gmail.com"
senha = "sua_senha_da_betfair"
app_key = "sua_key_gerada_no_video_anterior"

In [ ]:
# Carrega as variáveis do arquivo .env
# Essa célula deve ser usada apenas no Jupyter!!!

dotenv.load_dotenv()
username = os.getenv("EMAIL")
pw = os.getenv("SENHA")
app_key = os.getenv("APP_KEY")
print(email)

In [ ]:

#Abrindo a conexão com a lib betfairlightweight
trading = betfairlightweight.APIClient(username, pw, app_key=app_key, cert_files=('./FutBetBot.crt','./FutBetBot.pem'))
trading.login()

# Cria lista de jogos do dia

In [ ]:
def buscar_jogos_futebol(trading):
    # Lista para acumular os dados dos eventos
    dados_eventos = []

    # Definindo os filtros de Mercado para competições de futebol
    filtros_competicoes = betfairlightweight.filters.market_filter(
        event_type_ids=['1'],  # ID para Futebol
        market_start_time={'to': (datetime.utcnow() + timedelta(days=2)).strftime("%Y-%m-%dT%TZ")},
        #competition_ids = id_list # Competições
        # ***************************************************************************
        # Aqui dá para inserir mais um filtro para selecionar apenas as ligas de interesse.
        # ***************************************************************************
    )

    # Listando as competições de futebol disponíveis
    competicoes_futebol = trading.betting.list_competitions(filter=filtros_competicoes)

    # Laço para rodar todas as competições e pegar eventos
    for event_object in competicoes_futebol:
        # Cria filtro para buscar jogos de competições ativas
        filtros_jogos = betfairlightweight.filters.market_filter(
            event_type_ids=['1'],
            competition_ids=[int(event_object.competition.id)],
            market_start_time={'to': (datetime.utcnow() + timedelta(days=2)).strftime("%Y-%m-%dT%TZ")}
        )

        # Listando os eventos de futebol
        eventos_futebol = trading.betting.list_events(filter=filtros_jogos)

        # Acumula os dados de cada evento
        for obj_evento in eventos_futebol:
            dados_eventos.append({
                'DataAbertura': obj_evento.event.open_date,
                'IDEvento': obj_evento.event.id,
                'NomeEvento': obj_evento.event.name,
                'Liga': event_object.competition.name,
                'CodPais': obj_evento.event.country_code,
                'TotalMercados': obj_evento.market_count
            })

    # Cria DataFrame a partir da lista de dados
    tabela_jogos = pd.DataFrame(dados_eventos)
    return tabela_jogos

tabela_jogos = buscar_jogos_futebol(trading)
tabela_jogos.head(10)

,DataAbertura,IDEvento,NomeEvento,Liga,CodPais,TotalMercados
0,2024-05-29 16:53:00,97381,Specials,Specials,None,2
1,2024-06-22 23:00:00,33354156,Miami FC v Loudoun United FC,US United Soccer League,US,24
2,2024-06-23 00:00:00,33354157,Louisville FC v Rhode Island FC,US United Soccer League,US,24
3,2024-06-23 00:30:00,33354158,Tulsa Roughnecks FC v Sacramento Republic,US United Soccer League,US,24
4,2024-06-22 22:00:00,33354153,Monterey Bay FC v Oakland Roots,US United Soccer League,US,24
5,2024-06-23 02:30:00,33354169,Las Vegas Lights FC v Memphis 901 FC,US United Soccer League,US,24
6,2024-06-23 01:00:00,33354164,New Mexico United v Colorado Springs,US United Soccer League,US,24
7,2024-06-22 23:30:00,33354165,North Carolina FC v Pittsburgh Riverhounds,US United Soccer League,US,24
8,2024-06-22 23:00:00,33354151,Indy Eleven v Orange County Blues,US United Soccer League,US,24
9,2024-06-21 23:30:00,33354160,Hartford Athletic FC v Tampa Bay Rowdies,US United Soccer League,US,24


## Lista os mercados disponíveis em uma lista de jogos

In [ ]:
def buscar_mercados_por_evento(trading, tabela_jogos):
    # Lista para armazenar os dados dos mercados
    dados_mercados = []

    # Lista de mercados monitorados para filtragem
    mercados_monitorados = [
        'Both teams to Score?',
        'First Half Goals 0.5',
        'First Half Goals 1.5',
        'First Half Goals 2.5',
        'Half Time',
        'Match Odds',
        'Over/Under 0.5 Goals',
        'Over/Under 1.5 Goals',
        'Over/Under 2.5 Goals',
        'Over/Under 3.5 Goals',
        'Over/Under 4.5 Goals',
        'Over/Under 5.5 Goals',
        'Over/Under 6.5 Goals'
    ]

    # Itera sobre cada linha em tabela_jogos
    for _, row in tabela_jogos.iterrows():
        id_evento = row['IDEvento']
        nome_evento = row['NomeEvento']

        # Cria filtro para buscar mercados do evento específico
        filtro_catalogo_mercados = betfairlightweight.filters.market_filter(event_ids=[id_evento])

        # Busca os mercados para o evento atual
        catalogos_mercado = trading.betting.list_market_catalogue(
            filter=filtro_catalogo_mercados,
            max_results='100',
            sort='FIRST_TO_START',
            market_projection=['RUNNER_METADATA']
        )

        # Acumula os dados dos mercados
        for market_cat_object in catalogos_mercado:
            if market_cat_object.market_name in mercados_monitorados:
                dados_mercados.append({
                    'IDEvento': id_evento,
                    'NomeEvento': nome_evento,
                    'NomeMercado': market_cat_object.market_name,
                    'IDMercado': market_cat_object.market_id,
                    'TotalCorrespondido': market_cat_object.total_matched,
                    'Home': market_cat_object.runners[0].runner_name if len(market_cat_object.runners) > 0 else '',
                    'Home_id': market_cat_object.runners[0].selection_id if len(market_cat_object.runners) > 0 else 0,
                    'Away': market_cat_object.runners[1].runner_name if len(market_cat_object.runners) > 1 else '',
                    'Away_id': market_cat_object.runners[1].selection_id if len(market_cat_object.runners) > 1 else 0,
                    'Draw': market_cat_object.runners[2].runner_name if len(market_cat_object.runners) > 2 else '',
                    'Draw_id': market_cat_object.runners[2].selection_id if len(market_cat_object.runners) > 2 else 0
                })
            else:
                pass

    # Cria DataFrame a partir da lista de dados
    planilha_mercados = pd.DataFrame(dados_mercados)
    return planilha_mercados

planilha_mercados = buscar_mercados_por_evento(trading, tabela_jogos)
planilha_mercados.head(20)

,IDEvento,NomeEvento,NomeMercado,IDMercado,TotalCorrespondido,Home,Home_id,Away,Away_id,Draw,Draw_id
0,33354156,Miami FC v Loudoun United FC,Match Odds,1.229984610,0.0000,Miami FC,3155358,Loudoun United FC,23254245,The Draw,58805
1,33354156,Miami FC v Loudoun United FC,Half Time,1.229984695,0.0000,Miami FC,3155358,Loudoun United FC,23254245,The Draw,58805
2,33354156,Miami FC v Loudoun United FC,Both teams to Score?,1.229984657,0.0000,Yes,30246,No,110503,,0
3,33354156,Miami FC v Loudoun United FC,First Half Goals 2.5,1.229984622,0.0000,Under 2.5 Goals,47972,Over 2.5 Goals,47973,,0
4,33354156,Miami FC v Loudoun United FC,First Half Goals 1.5,1.229984621,0.0000,Under 1.5 Goals,1221385,Over 1.5 Goals,1221386,,0
5,33354156,Miami FC v Loudoun United FC,First Half Goals 0.5,1.229984620,0.0000,Under 0.5 Goals,5851482,Over 0.5 Goals,5851483,,0
6,33354156,Miami FC v Loudoun United FC,Over/Under 6.5 Goals,1.229984617,0.0000,Under 6.5 Goals,2542448,Over 6.5 Goals,2542449,,0
7,33354156,Miami FC v Loudoun United FC,Over/Under 4.5 Goals,1.229984615,2693.0904,Under 4.5 Goals,1222347,Over 4.5 Goals,1222346,,0
8,33354156,Miami FC v Loudoun United FC,Over/Under 5.5 Goals,1.229984616,0.0000,Under 5.5 Goals,1485567,Over 5.5 Goals,1485568,,0
9,33354156,Miami FC v Loudoun United FC,Over/Under 3.5 Goals,1.229984614,0.0000,Under 3.5 Goals,1222344,Over 3.5 Goals,1222345,,0


In [ ]:
market_id='1.229984615'
# Obtenha as odds atuais

market_book = trading.betting.list_market_book(
    market_ids=[market_id]
)

# Imprimir as odds
for market in market_book:
    for runner in market.runners:
        print(f'Runner ID: {runner.selection_id}, Odds: {runner.last_price_traded}')

Runner ID: 1222347, Odds: 1.08
Runner ID: 1222346, Odds: 13.5


## Envia ordem para mercado escolhido

In [ ]:


def place_bet(trading, market_id, selection_id, bet_type, price, size):
    """
    Envia uma aposta para a BetFair usando o pacote betfairlightweight.

    :param trading: Instância autenticada do APIClient.
    :param market_id: ID do mercado onde a aposta será colocada.
    :param selection_id: ID da seleção na qual a aposta será colocada.
    :param bet_type: Tipo de aposta, 'BACK' ou 'LAY'.
    :param price: Preço (odds) da aposta.
    :param size: Tamanho da aposta (stake).
    """
    from betfairlightweight.resources.bettingresources import (
        LimitOrder,
        MarketOnCloseOrder,
        PlaceOrderInstruction
    )

    # Criar uma ordem de aposta
    if bet_type == 'MARKET_ON_CLOSE':
        order = MarketOnCloseOrder(liability=size)
    else:
        order = {
    "size": size,
    "price": price,
    "persistenceType": 'LAPSE'
}

    # Criar instrução de aposta
    instruction_dict = {
        "selectionId": selection_id,
        "handicap": "0",
        "side": "LAY",
        "orderType": bet_type,
        "limitOrder": order
    }

    # Enviar a aposta
    try:
        response = trading.betting.place_orders(market_id=market_id, instructions=[instruction_dict])
        print(response)
        return response
    except Exception as e:
        print(f"Erro ao colocar a aposta: {e}")
        return None

place_bet(trading, "1.229984615", "1222347", 'LIMIT', "1.08", "5.0")

## Confere se a ordem foi enviada

In [ ]:
#Confere que a ordem foi enviada

import datetime

cleared_orders = trading.betting.list_current_orders(
    #bet_status=bet_status,
    #event_ids= [event_ids],
    #market_ids=[market_id],
    #event_type_ids=event_type_ids
)
print(f'Código aposta: {cleared_orders._data["currentOrders"][0]["betId"]}')
print(f'Id do mercado: {cleared_orders._data["currentOrders"][0]["marketId"]}')
print(f'Id do resultado selecionado: {cleared_orders._data["currentOrders"][0]["selectionId"]}')
print(f'Valor da Odd: {cleared_orders._data["currentOrders"][0]["priceSize"]["price"]}')
print(f'Valor da aposta: {cleared_orders._data["currentOrders"][0]["priceSize"]["size"]}')


print(f'Hora de envio da aposta: {cleared_orders._data["currentOrders"][0]["placedDate"]}')